In [114]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from implicit import bpr
from implicit.cpu.lmf import LogisticMatrixFactorization
from implicit.cpu.als import AlternatingLeastSquares
from sklearn.linear_model import SGDClassifier
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import scipy
import scipy.optimize
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from scipy import sparse
from collections import defaultdict
from typing import List
import lightfm
import datetime
import os
import tensorflow.keras as keras
from tensorflow.keras.layers import (
    Concatenate,
    Dense,
    Embedding,
    Flatten,
    Input,
    Multiply,
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# Would Play

In [2]:
def readJSON(path):
    f = open(path)
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g

In [3]:
rawData = []
for l in readJSON("train.json"):
    rawData.append(l)

In [4]:
random.shuffle(rawData)
train = rawData[:165000]
valid = rawData[165000:]

In [5]:
usersPerGame = defaultdict(set)
gamesPerUser = defaultdict(set)
for user,game in rawData:
    usersPerGame[game].add(user)
    gamesPerUser[user].add(game)
games = list(usersPerGame.keys())
users = list(gamesPerUser.keys())

In [7]:
def mostPopularGames(data,threshold):
    gameCount = defaultdict(int)
    
    for user,game in data:
        gameCount[game] += 1
    
    mostPopular = [(gameCount[x], x) for x in gameCount]
    mostPopular.sort()
    mostPopular.reverse()
    
    mostPopularGames= set()
    notPopularGames = set()
    count = 0
    
    for ic, i in mostPopular:
        count += ic
        if count <= threshold:
            mostPopularGames.add(i)
        if count > threshold: 
            notPopularGames.add(i)#break
    return mostPopularGames,notPopularGames

In [6]:
gameCount = defaultdict(int)
for user,game in train:
    gameCount[game] += 1
    
    
mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()
gamerank = {}
gamePop = {}
for i in range(len(mostPopular)):
    gamerank[mostPopular[i][1]] = i+1
    gamePop[mostPopular[i][1]] = (int(mostPopular[i][0])/mostPopular[0][0])
#gamePop

In [8]:
newValid = []
falseSamples = defaultdict(list)
for u,g in valid:
    newValid.append([u,g,1])
    res1 = random.sample([ele for ele in games if ele not in list(gamesPerUser[u])+falseSamples[u]],1)
    falseSamples[u].extend(res1)
    newValid.append([u,res1[0],0])

In [9]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [10]:
def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [11]:
usersPerGameTrain = defaultdict(set)
gamesPerUserTrain = defaultdict(set)
for user,game in train:
    usersPerGameTrain[game].add(user)
    gamesPerUserTrain[user].add(game)
gamesTrain = list(usersPerGameTrain.keys())
usersTrain = list(gamesPerUserTrain.keys())

In [12]:
max_sims = []
for i in range(len(newValid)):
    u = newValid[i][0]
    g = newValid[i][1]
    gamesByUser = gamesPerUserTrain[u]
    users = usersPerGameTrain[g]
    max_sim = 0
    
    for g_ in gamesByUser:
        if (g_==g): continue
        sim = Jaccard(users, usersPerGameTrain[g_])
        max_sim = max(max_sim,sim)

    max_sims.append(max_sim)

In [13]:
userIDs,itemIDs = {},{}

for u,i in train:
    #u,i = d['user_id'],d['book_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers,nItems = len(userIDs),len(itemIDs)

In [14]:
Xui = scipy.sparse.lil_matrix((nUsers, nItems))
for u,g in train:
    Xui[userIDs[u],itemIDs[g]] = 1
    
Xui_csr = scipy.sparse.csr_matrix(Xui)

In [15]:
model2 = bpr.BayesianPersonalizedRanking(factors = 3)
model2.fit(Xui_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
bpr_score = []
for u,i,l in newValid:
    print(u)
    if(u not in userIDs):
        bpr_score.append(0)
        continue
    uid = userIDs[u]
    ids, scores = model2.recommend(uid, Xui_csr[uid],N=nItems)
    #print(scores)
    s = scores[list(ids).index(itemIDs[i])]
    bpr_score.append(s)
    #break

In [17]:
model3 = LogisticMatrixFactorization(factors = 3)
model3.fit(Xui_csr)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
lmf_score = []
for u,i,l in newValid:
    print(u)
    if(u not in userIDs):
        lmf_score.append(0)
        continue
    uid = userIDs[u]
    ids, scores = model3.recommend(uid, Xui_csr[uid],N=nItems)
    s = scores[list(ids).index(itemIDs[i])]
    lmf_score.append(s)

In [19]:
mpg,npg = mostPopularGames(train,len(train)*0.75)
features = []
y = []
for i in range(len(newValid)):
    y.append(newValid[i][2])
    
    temp = [1]
#     if newValid[i][1] in mpg:
#         temp.append(1)
#     else:
#         temp.append(0)
        
    temp.append(max_sims[i])
    #temp.append(gamerank[newValid[i][1]])
    temp.append(gamePop[newValid[i][1]])
    temp.append(bpr_score[i])
    temp.append(lmf_score[i])
    features.append(temp)

In [20]:
#model = linear_model.LogisticRegression(C=1,max_iter=500)
model1 = GradientBoostingClassifier()
#model1 = SGDClassifier()
model1.fit(features,y)
valid_pred = model1.predict(features)
accuracy = sum(valid_pred==y)/len(y)
accuracy

0.7384238423842384

In [21]:
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    gamesByUser = gamesPerUserTrain[u]
    users = usersPerGameTrain[g]
    max_sim = 0

    
    for g_ in gamesByUser:
        if (g_==g): continue
        sim = Jaccard(users, usersPerGameTrain[g_])
        max_sim = max(max_sim,sim)
        
    if(u not in userIDs):
        bs=0#sum(bpr_score)/len(bpr_score)
    else:
        uid = userIDs[u]
        ids, scores = model2.recommend(uid, Xui_csr[uid],N=nItems)
        bs = scores[list(ids).index(itemIDs[g])]
        
    if(u not in userIDs):
        lmf=0#sum(lmf_score)/len(lmf_score)
    else:
        uid = userIDs[u]
        ids, scores = model3.recommend(uid, Xui_csr[uid],N=nItems)
        lmf = scores[list(ids).index(itemIDs[g])]
        
    # Logic...
    if g in mpg:
        a=1
    else:
        a=0
    predV = model1.predict([[1,max_sim,gamePop[g],bs,lmf]])[0]
    #,gamePop[g],bs,lmf

    #model.coef_[0][0]+model.coef_[0][1]*a+model.coef_[0][2]*max_sim #>0 ? 1:0
    #print(predV)

    #print(pred)
    
    _ = predictions.write(u + ',' + g + ',' + str(predV) + '\n')

predictions.close()

# Hours Played

### Implementation from HW3 - No regularizer

In [40]:
def readJSON(path):
    #f = gzip.open(path, 'rt')
    f = open(path)
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        h = d['hours_transformed']
        yield u,g,h

In [41]:
allHours = []
for l in readJSON("train.json"):
    allHours.append(l)

In [42]:
random.shuffle(allHours)
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [43]:
trainHours = [r[2] for r in hoursTrain]
globalAverage = sum(trainHours) * 1 / len(trainHours)

In [44]:
hoursPerItem = defaultdict(list)
hoursPerUser = defaultdict(list)
for user,game,info in hoursTrain:
    hoursPerItem[game].append(info)
    hoursPerUser[user].append(info)

In [45]:
usersPerGame = defaultdict(list)
gamesPerUser = defaultdict(list)
for user,game,info in hoursTrain:
    usersPerGame[game].append(user)
    gamesPerUser[user].append(game)

In [46]:
def iterate(lamb,iter):
    betaU = {}
    betaI = {}
    for u in hoursPerUser:
        betaU[u] = 0

    for g in hoursPerItem:
        betaI[g] = 0
    
    alpha = globalAverage 
    Ntrain = len(hoursTrain)

    for i in range(iter):
        tempAlpha = 0
        for user,game,info in hoursTrain:
            tempAlpha += info - betaU[user] - betaI[game]
        tempAlpha = tempAlpha/Ntrain
        
        alpha = tempAlpha
        #print(alpha)
        
        for u in hoursPerUser:
            sizeIu = len(gamesPerUser[u])
            tempbetaU = sum(hoursPerUser[u]) - sizeIu*alpha
            for  g in gamesPerUser[u]:
                tempbetaU -= betaI[g]
            tempbetaU = tempbetaU/(sizeIu+lamb)
            betaU[u] = tempbetaU
        
        for g in hoursPerItem:
            sizeIg = len(usersPerGame[g])
            tempbetaI = sum(hoursPerItem[g]) - sizeIg*alpha
            for  u in usersPerGame[g]:
                tempbetaI -= betaU[u]
            tempbetaI = tempbetaI/(sizeIg+lamb)
            betaI[g] = tempbetaI
            
        if((i+1)%10==0):
            print("Iteration: " + str(i+1))
            print("Alpha: " + str(alpha))
            preds = []
            for u,g,_ in hoursValid:
                preds.append(alpha+betaU[u]+betaI[g])
            y = [r[2] for r in hoursValid]
            validMSE = numpy.mean(numpy.square(numpy.array(y) - numpy.array(preds)))
            print("MSE: "+ str(validMSE))
            print("******************************")
        
    return alpha,betaU,betaI

In [47]:
alpha,betaU,betaI = iterate(5,70)

Iteration: 10
Alpha: 3.4569930412531455
MSE: 3.061680547939428
******************************
Iteration: 20
Alpha: 3.294122093450818
MSE: 3.059591560764978
******************************
Iteration: 30
Alpha: 3.2093851667267566
MSE: 3.0586832055684043
******************************
Iteration: 40
Alpha: 3.1654082326684656
MSE: 3.0582610592691992
******************************
Iteration: 50
Alpha: 3.142603580029856
MSE: 3.0580555448862405
******************************
Iteration: 60
Alpha: 3.130781183415017
MSE: 3.0579526207798597
******************************
Iteration: 70
Alpha: 3.124652753490682
MSE: 3.057900242717601
******************************


In [48]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    # Logic...
    pred = alpha+betaU[u]+betaI[g]
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

### Scipy optimization

In [145]:
def readJSON2(path):
    f = open(path)
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        h = d['hours_transformed']
        yield u,g,h
        

In [146]:
allHours = []
for l in readJSON2("train.json"):
    allHours.append(l)

In [147]:
def trainValidSplit(datals):

    random.shuffle(datals)
    data = pd.DataFrame(datals,columns = ['userID','gameID','hours'])
    train, valid = data[:165000], data[165000:]

    return data, train, valid

In [148]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))
    
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d['userID'], d['gameID']) for index, d in train.iterrows()]
    cost = MSE(predictions, labels)
    #print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    print("Cost = " + str(cost))
    return cost

def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(train)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for index, d in train.iterrows():
        u,i = d['userID'], d['gameID']
        pred = prediction(u, i)
        diff = pred - d['hours']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)

In [159]:
lamb = 0.0000275
data, train, valid = trainValidSplit(allHours)

ratingMean = train['hours'].mean()
alpha = ratingMean

labels = train['hours']

userBiases = defaultdict(float)
itemBiases = defaultdict(float)

users = list(set(train['userID']))
items = list(set(train['gameID']))
nUsers = len(users)
nItems = len(items)
#fmin_l_bfgs_b
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0]*(nUsers+nItems),
                             derivative, args = (labels, lamb),maxiter=20)

Cost = 5.290959200480984
Cost = 5.172056515137862
Cost = 4.763495442850006
Cost = 15.545584592513642
Cost = 4.570176495541972
Cost = 4.087647232371297
Cost = 4.074834646145797
Cost = 4.02536025443987
Cost = 3.8558779387483657
Cost = 3.531735949981736
Cost = 3.3591163189760156
Cost = 3.1752274370865297
Cost = 3.1031109236446763
Cost = 3.0254233089299363
Cost = 2.982780278250415
Cost = 2.9529674236876136
Cost = 2.9418800135168857
Cost = 2.9327184300120455
Cost = 4.81646866755446
Cost = 2.932671094720736
Cost = 2.929367391021621
Cost = 2.926363354501926
Cost = 2.924094753579522
Cost = 2.922192751511797
Cost = 2.920115710730171
Cost = 2.9181272200855153


(array([ 3.11811897,  0.34117967,  0.7521703 , ...,  0.42361739,
        -0.17546085,  3.25501628]),
 2.9181272200855153,
 {'grad': array([-6.97385435e-04, -2.27187960e-06, -2.90026926e-05, ...,
          2.45444423e-06, -9.71493554e-07, -1.87175843e-05]),
  'task': 'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT',
  'funcalls': 26,
  'nit': 20,
  'warnflag': 1})

In [160]:
predictions = []
for index, d in valid.iterrows():
    u, i = d['userID'], d['gameID']
    if u in userBiases and i in itemBiases:
        predictions.append(prediction(u, i))
    else:
        predictions.append(0)

print("MSE %.3f" % MSE(predictions, valid['hours']))

MSE 2.869


In [161]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    if u in userBiases and g in itemBiases:
        pred = prediction(u, g)
    else:
        pred = 0
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

### Tensorflow LFM

In [163]:
userIDs = {}
itemIDs = {}
interactions = []
f = open("train.json")
f.readline()

for l in f:
    d = eval(l)
    u = d['userID']
    i = d['gameID']
    r = d['hours_transformed']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

    interactions.append((u,i,r))

In [164]:
random.shuffle(interactions)
len(interactions)

174999

In [165]:
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [166]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

In [167]:
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)

In [168]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [169]:
class LatentFactorModelBiasOnly(tf.keras.Model):
    def __init__(self, mu, lamb):
        super(LatentFactorModelBiasOnly, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i]
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        pred = self.alpha + beta_u + beta_i
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [170]:
def trainingStepBiasOnly(model, interactionsTrain):
    Nsamples = 20000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactionsTrain)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
        (grad, var) in zip(gradients, model.trainable_variables)
        if grad is not None)
    return loss.numpy()

In [171]:
optimizer = tf.keras.optimizers.legacy.Adam(0.1)
modelBiasOnly = LatentFactorModelBiasOnly(mu, 0.000025)
for i in range(60):
    obj = trainingStepBiasOnly(modelBiasOnly, interactionsTrain)
    if (i % 10 == 9): 
        print("iteration " + str(i+1) + ", objective = " + str(obj))
    if (i % 10 == 9):    
        biasOnlyPredictions =[modelBiasOnly.predict(userIDs[u],itemIDs[i]).numpy() for u,i,_ in interactionsTest]
        labels = [r for _,_,r in interactionsTest]
        
        print(MSE(biasOnlyPredictions, labels))

iteration 10, objective = 1.903913
3.72244084054001
iteration 20, objective = 1.6490375
3.2941192996675004
iteration 30, objective = 1.5630397
3.1529835833335356
iteration 40, objective = 1.5568713
3.0905484621327126
iteration 50, objective = 1.5434971
3.066342441437411
iteration 60, objective = 1.5824833
3.058982698534239


In [172]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
     
    if u in userIDs and g in itemIDs:
        pred = modelBiasOnly.predict(userIDs[u],itemIDs[g]).numpy()
    else:
        pred = mu
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

### Light FM with early stopping

In [83]:

def readJSON(path):
    #f = gzip.open(path, 'rt')
    f = open(path)
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        h = d['hours_transformed']
        yield u,g,h

In [84]:
userIDs = {}
gameIDs = {}

f = open("train.json")
f.readline()
for l in f:
    d = eval(l)
    u = d['userID']
    g = d['gameID']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in gameIDs: gameIDs[g] = len(gameIDs)

In [132]:
rawData = []
for l in readJSON("train.json"):
    rawData.append(l)
random.shuffle(rawData)

In [133]:
usersPerGame = defaultdict(set)
gamesPerUser = defaultdict(set)
for user,game,_ in rawData:
    usersPerGame[game].add(user)
    gamesPerUser[user].add(game)
games = list(usersPerGame.keys())
users = list(gamesPerUser.keys())

In [134]:
nTrain = int(len(rawData) * 0.95)
nTest = len(rawData) - nTrain
train = rawData[:nTrain]
test = rawData[nTrain:]

In [135]:
df_train = pd.DataFrame(train, columns=["user", "item", "interaction"])
df_train["user_id"] = df_train["user"].map(userIDs)
df_train["item_id"] = df_train["item"].map(gameIDs)
df_train = df_train[["user_id", "item_id", "interaction"]]
n_users = df_train['user_id'].nunique()
n_items = df_train['item_id'].nunique()

In [136]:
def create_ncf(
    number_of_users: int,
    number_of_items: int,
    latent_dim_mf: int = 2,
    latent_dim_mlp: int = 2,
    reg_mf: int = 0.001,
    reg_mlp: int = 0.002,
    dense_layers: List[int] = [16, 8],
    reg_layers: List[int] = [0.01, 0.01],
) -> keras.Model:

    # input layer
    user = Input(shape=(), dtype="int32", name="user_id")
    item = Input(shape=(), dtype="int32", name="item_id")

    # embedding layers
    mf_user_embedding = Embedding(
        input_dim=number_of_users,
        output_dim=latent_dim_mf,
        name="mf_user_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mf),
        input_length=1,
    )
    mf_item_embedding = Embedding(
        input_dim=number_of_items,
        output_dim=latent_dim_mf,
        name="mf_item_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mf),
        input_length=1,
    )

    mlp_user_embedding = Embedding(
        input_dim=number_of_users,
        output_dim=latent_dim_mlp,
        name="mlp_user_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mlp),
        input_length=1,
    )
    mlp_item_embedding = Embedding(
        input_dim=number_of_items,
        output_dim=latent_dim_mlp,
        name="mlp_item_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mlp),
        input_length=1,
    )

    # MF vector
    mf_user_latent = Flatten()(mf_user_embedding(user))
    mf_item_latent = Flatten()(mf_item_embedding(item))
    mf_cat_latent = Multiply()([mf_user_latent, mf_item_latent])

    # MLP vector
    mlp_user_latent = Flatten()(mlp_user_embedding(user))
    mlp_item_latent = Flatten()(mlp_item_embedding(item))
    mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])

    mlp_vector = mlp_cat_latent

    # build dense layers for model
    for i in range(len(dense_layers)):
        layer = Dense(
            dense_layers[i],
            activity_regularizer=l2(reg_layers[i]),
            name="layer%d" % i,
        )
        mlp_vector = layer(mlp_vector)

    predict_layer = Concatenate()([mf_cat_latent, mlp_vector])

    result = Dense(
        1, name="interaction"
    )

    output = result(predict_layer)

    model = Model(
        inputs=[user, item],
        outputs=[output],
    )

    return model

In [137]:
def make_tf_dataset(df: pd.DataFrame,targets: List[str],val_split: float = 0.05,batch_size: int = 4096,seed=None,):

    n_val = round(df.shape[0] * val_split)
    if seed:
        # shuffle all the rows
        x = df.sample(frac=1, random_state=seed).to_dict("series")
    else:
        x = df.to_dict("series")
    y = dict()
    for t in targets:
        y[t] = x.pop(t)
    ds = tf.data.Dataset.from_tensor_slices((x, y))

    ds_val = ds.take(n_val).batch(batch_size)
    ds_train = ds.skip(n_val).batch(batch_size)
    
    return ds_train, ds_val

In [138]:
ncf_model = create_ncf(n_users, n_items)

ncf_model.compile(
    optimizer=Adam(),
    #loss="binary_crossentropy",
    loss="mean_squared_error",
    metrics=[
            tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error')
    ],
)
ncf_model._name = "neural_collaborative_filtering"

In [139]:
ds_train, ds_val = make_tf_dataset(df_train, ["interaction"])
N_EPOCHS = 30
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_root_mean_squared_error", patience=1
)

train_hist = ncf_model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=N_EPOCHS,
    callbacks=[tensorboard_callback, early_stopping_callback],
    verbose=1,
)

Epoch 1/30
39/39 [==============================] - 1s 7ms/step - loss: 17.7503 - root_mean_squared_error: 4.2014 - val_loss: 16.0379 - val_root_mean_squared_error: 3.9950
Epoch 2/30
39/39 [==============================] - 0s 6ms/step - loss: 13.0083 - root_mean_squared_error: 3.5936 - val_loss: 9.8855 - val_root_mean_squared_error: 3.1234
Epoch 3/30
39/39 [==============================] - 0s 6ms/step - loss: 7.0535 - root_mean_squared_error: 2.6216 - val_loss: 5.2788 - val_root_mean_squared_error: 2.2468
Epoch 4/30
39/39 [==============================] - 0s 6ms/step - loss: 4.6611 - root_mean_squared_error: 2.1013 - val_loss: 4.4039 - val_root_mean_squared_error: 2.0387
Epoch 5/30
39/39 [==============================] - 0s 6ms/step - loss: 3.9965 - root_mean_squared_error: 1.9365 - val_loss: 3.8818 - val_root_mean_squared_error: 1.9063
Epoch 6/30
39/39 [==============================] - 0s 8ms/step - loss: 3.5513 - root_mean_squared_error: 1.8167 - val_loss: 3.5516 - val_root_mean

In [140]:
df_test = pd.DataFrame(test, columns=["user", "item", "interaction"])
df_test["user_id"] = df_test["user"].map(userIDs)
df_test["item_id"] = df_test["item"].map(gameIDs)
df_test=df_test[["user_id", "item_id", "interaction"]]
ds_test, _ = make_tf_dataset(df_test, ["interaction"], val_split=0, seed=None)
ncf_predictions = ncf_model.predict(ds_test)
df_test["ncf_predictions"] = ncf_predictions

3/3 [==============================] - 0s 3ms/step


In [141]:
np.mean(np.square(np.array(df_test['interaction']) - np.array(df_test['ncf_predictions'])))

3.0314861250886445

In [142]:
dfc = pd.read_csv("pairs_Hours.csv")
dfc["user_id"] = dfc["userID"].map(userIDs)
dfc["item_id"] = dfc["gameID"].map(gameIDs)
dfc["interaction"] = dfc["prediction"]
dfc=dfc[["user_id", "item_id", "interaction"]]
dsc, _ = make_tf_dataset(dfc, ["interaction"], val_split=0, seed=None)
preds = ncf_model.predict(dsc)
dfc["ncf_predictions"] = preds

3/3 [==============================] - 0s 4ms/step


In [143]:
dfc

,user_id,item_id,interaction,ncf_predictions
0,1374,459,NaN,4.272638
1,5565,590,NaN,0.990524
2,759,281,NaN,5.285567
3,1928,784,NaN,2.951291
4,6216,600,NaN,3.294409
...,...,...,...,...
9995,4630,392,NaN,6.995924
9996,5520,481,NaN,3.865772
9997,2930,1889,NaN,3.213949
9998,892,1728,NaN,3.620036


In [144]:
sample = dfc.values.tolist()
i=0
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    pred = sample[i][3]
    #print(sample[i])
    i = i+1
    
    #print(pred)
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()